# Test suite for Project 1


In [13]:
from proj1_helpers import *
import pickle

## Data loading

* `../data/train.csv`
* `../data/test.csv`


In [14]:
train_path = '../data/train.csv'
test_path  = '../data/test.csv'

In [15]:
# Long, run only if necessary
train_data = load_csv_data(train_path)

In [16]:
# Long, run only if necessary
test_data = load_csv_data(test_path)

In [17]:
# Save the data as pickles for more efficient reloading
# Only run once to generate the pickle !
# pickle.dump(test_data, open( 'test.p', 'wb' ))
# pickle.dump(train_data, open( 'train.p', 'wb' ))

In [19]:
# loading the pickle back 
# train_data = pickle.load(open( 'train.p', 'rb' ))
# test_data = pickle.load(open( 'test.p', 'rb' ))

## Exploratory

* Describe / Discover the data
* Describe / Characterise the features

## Data processing

* Split test / train
* Clean useless features
* [Scale / process data](https://machinelearningmastery.com/discover-feature-engineering-how-to-engineer-features-and-how-to-get-good-at-it/) (e.g. scaling)

## Training

* **Train** the model with the different learning algorithms
    * `least_squares`
    * `least_squares_GD`
    * `least_squares_SGD`
    * `ridge_regression`
    * `logistic_regression`
    * `reg_logistic_regression`

## Testing

* **Test** the model with the weights computed from the different learning algorithms to find the best one

## Cross-validate

* the hyperparameters for each algorithm
    * `least_squares` 
    * `least_squares_GD` -> `gamma`
    * `least_squares_SGD` -> `gamma`, `batch_size`
    * `ridge_regression` -> `lambda_`
    * `logistic_regression` -> `gamma`
    * `reg_logistic_regression` -> `lambda_`, `gamma`
* Also, if we use other features (e.g. polynomial), we should CV those as well

## Results

* Plot the train / test accuracies for the best set of algorithm + parameters